In [22]:
import cv2
import os

# video-path : Chemin de la vidéo à traiter
# output_folder : Dossier de destination
def video_to_frames(video_path, output_folder, frames_number):    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Chargement de la vidéo
    video_capture = cv2.VideoCapture(video_path)

    # Calcul du pas entre chaque frame pour obtenir le nombre requis
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_indices = [int(i * (total_frames - 1) / (frames_number - 1)) for i in range(frames_number)]

    # Lecture et extraction des frames
    count = 0  # Compteur pour nommer les frames
    extracted_frames = 0

    while True:
        success, image = video_capture.read()

        if not success:
            break

        if count in frames_indices:
            frame_path = os.path.join(output_folder, f"frame{extracted_frames}.jpg")
            cv2.imwrite(frame_path, image)
            extracted_frames += 1

        count += 1

    # Libération de la ressource vidéo
    video_capture.release()

In [23]:

output_folder1 = 'Personne_qui_marche/images'
video_path1 = 'Personne_qui_marche/Personne_qui_marche.mp4'

video_to_frames(video_path1, output_folder1, 10)

In [4]:
import ast
import os.path
from xml.dom import minidom

out_dir = './out'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

file = minidom.parse('annotations.xml')

images = file.getElementsByTagName('image')

for image in images:
    width = int(image.getAttribute('width'))
    height = int(image.getAttribute('height'))
    name = image.getAttribute('name')
    elem = image.getElementsByTagName('points')
    boxes = image.getElementsByTagName('box')  # Obtenez toutes les balises box pour cette image

    if len(boxes) > 0:  # Vérifiez s'il y a des balises box pour cette image
        bbox = boxes[0]  # Prenez la première balise box
        xtl = int(float(bbox.getAttribute('xtl')))
        ytl = int(float(bbox.getAttribute('ytl')))
        xbr = int(float(bbox.getAttribute('xbr')))
        ybr = int(float(bbox.getAttribute('ybr')))
        w = xbr - xtl
        h = ybr - ytl
        label_file = open(os.path.join(out_dir, name[:-4] + '.txt'), 'w')

        for e in elem:
            label_file.write('0 {} {} {} {} '.format(str((xtl + (w / 2)) / width), str((ytl + (h / 2)) / height),
                                                     str(w / width), str(h / height)))

            points = e.attributes['points']
            points = points.value.split(';')
            points_ = []
            for p in points:
                p = p.split(',')
                p1, p2 = p
                points_.append([int(float(p1)), int(float(p2))])
            for p_, p in enumerate(points_):
                label_file.write('{} {}'.format(p[0] / width, p[1] / height))
                if p_ < len(points_) - 1:
                    label_file.write(' ')
                else:
                    label_file.write('\n')

In [20]:
from ultralytics import YOLO


model = YOLO('yolov8n-pose.pt')  # load a pretrained model (recommended for training)

model.train(data='config.yaml', epochs=1, imgsz=640)


Ultralytics YOLOv8.0.215 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx)
engine\trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=config.yaml, epochs=1, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=F

train: Scanning C:\Users\boqua\ProjetM1\PoseEstimationFeatures\Personne_qui_marche\data\labels\train... 3 images, 0 bac

train: New cache created: C:\Users\boqua\ProjetM1\PoseEstimationFeatures\Personne_qui_marche\data\labels\train.cache



val: Scanning C:\Users\boqua\ProjetM1\PoseEstimationFeatures\Personne_qui_marche\data\labels\val... 2 images, 0 backgro

val: New cache created: C:\Users\boqua\ProjetM1\PoseEstimationFeatures\Personne_qui_marche\data\labels\val.cache


...

        1/1         0G      2.122      8.474          0      1.277       2.02          8        640: 100%|██████████| 1
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP


                   all          2          2      0.982          1      0.995      0.895          0          0          0          0

1 epochs completed in 0.005 hours.
Optimizer stripped from runs\pose\train11\weights\last.pt, 6.6MB
Optimizer stripped from runs\pose\train11\weights\best.pt, 6.6MB

Validating runs\pose\train11\weights\best.pt...
Ultralytics YOLOv8.0.215 🚀 Python-3.11.4 torch-2.1.1+cpu CPU (AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx)
YOLOv8n-pose summary (fused): 187 layers, 3190199 parameters, 0 gradients, 8.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP


                   all          2          2      0.982          1      0.995      0.895          0          0          0          0
Speed: 6.0ms preprocess, 290.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\pose\train11


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002B0D9D37CD0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,

In [6]:
import xml.etree.ElementTree as ET
import json

def convert_xml_to_coco_keypoints(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    images = []

    for image in root.findall('image'):
        image_info = {
            "image_id": int(image.get('id')),
            "file_name": image.get('name'),
            "width": int(image.get('width')),
            "height": int(image.get('height')),
            "annotations": []
        }

        for points in image.findall('points'):
            keypoints = []
            for point in points.get('points').split(';'):
                x, y = point.split(',')
                keypoints.extend([float(x), float(y), 2])  # Adding keypoint information (x, y, visibility)

            annotation = {
                "keypoints": keypoints,
                "category_id": 1,  # Assuming there's only one category 'Human'
                "iscrowd": 0
            }

            box = image.find('box')
            if box is not None:
                bbox_attrs = ['xtl', 'ytl', 'xbr', 'ybr']
                annotation["bbox"] = [float(box.get(attr)) for attr in bbox_attrs]

            image_info["annotations"].append(annotation)

        images.append(image_info)

    coco_format = {
        "images": images,
        "categories": [{"id": 1, "name": "Human", "supercategory": "person"}]
    }

    with open('output_coco.json', 'w') as outfile:
        json.dump(coco_format, outfile, indent=4)

# Utilisation de la fonction avec le fichier XML en entrée
convert_xml_to_coco_keypoints('annotations.xml')

In [19]:
import xml.etree.ElementTree as ET
import os

def convert_xml_to_yolo_keypoints(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for image in root.findall('image'):
        file_name = image.attrib['name']
        width = float(image.attrib['width'])
        height = float(image.attrib['height'])

        with open(f"{os.path.splitext(file_name)[0]}.txt", 'w') as txt_file:
            points = image.find('points')
            label = points.attrib['label']
            if label == 'Human':
                label = '0'
            keypoints = points.attrib['points'].split(';')

            xtl = float(image.find('box').attrib['xtl'])
            ytl = float(image.find('box').attrib['ytl'])
            xbr = float(image.find('box').attrib['xbr'])
            ybr = float(image.find('box').attrib['ybr'])

            # Normalizing bounding box coordinates
            x = (xtl + xbr) / (2 * width)
            y = (ytl + ybr) / (2 * height)
            bbox_width = abs(xtl - xbr) / width
            bbox_height = abs(ytl - ybr) / height

            # Extracting and normalizing keypoints
            normalized_keypoints = []
            for point in keypoints:
                coords = point.split(',')
                for coord in coords:
                    try:
                        coord = float(coord)
                        if len(normalized_keypoints) % 2 == 0:  # For x-coordinates
                            normalized_keypoints.append(coord / width)
                        else:  # For y-coordinates
                            normalized_keypoints.append(coord / height)
                    except ValueError:
                        pass

            # YOLO format: class index, x, y, width, height, keypoints
            txt_file.write(f"{label} {x} {y} {bbox_width} {bbox_height} {' '.join(map(str, normalized_keypoints))}\n")

# Utilisation de la fonction avec le fichier XML en entrée
convert_xml_to_yolo_keypoints('annotations.xml')
